# Creación del Pipeline con RandomForest

**Juan Leonardo Rangel**  
**Santiago Molina**  
**François Morales**


### Importar cositas

In [307]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

import pandas as pd
import numpy as np
import sys

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report

from scipy import stats as st
from nltk.stem import SnowballStemmer
from statistics import mode
from sklearn.base import BaseEstimator, ClassifierMixin

import re

import json
from sklearn.preprocessing import FunctionTransformer
import joblib

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leonardorangel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/leonardorangel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leonardorangel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Cargar el archivo excel con los datos y hacer split

In [309]:
data=pd.read_excel("ODScat_345.xlsx")
data_t, data_v = train_test_split(data, test_size=0.2, random_state=0)

In [310]:
data_t.head()

,Textos_espanol,sdg
2651,La movilización de las mujeres en manifestacio...,5
169,El papel combinado como comprador y proveedor ...,3
2993,El desempleo es particularmente alto entre los...,5
1148,"Además, el espíritu empresarial es un vehículo...",4
2270,Aunque Sudáfrica tenía una Oficina de la Condi...,5


### Funciones para el procesamiento del texto

In [312]:
def quitarPuntuacion(words):
    new_words = []
    for word in words:
        if word is not None:
            # Adjusted regular expression pattern to exclude colon
            new_word = re.sub(r'[^\w\s:]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

def aMinuscula(words):
    new_words = []
    for word in words:
        if word is not None:
            new_word = word.lower()
            if new_word != ' ':
                new_words.append(new_word)
    return new_words


def eliminarNumeros(words):
    new_words = []
    for word in words:
        if not contieneNumero(word):
            new_words.append(word)
    return new_words

def contieneNumero(s):
    pattern = re.compile(r'\d')
    return bool(pattern.search(s))


spanish_stopwords = set(stopwords.words('spanish'))
def quitarStopwords(words):
    new_words = []
    for word in words:
        if word is not None:
            if word not in spanish_stopwords:
                new_words.append(word)
    return new_words

def preProcesamiento(words):
    words = aMinuscula(words)
    words = eliminarNumeros(words)
    words = quitarPuntuacion(words)
    words = quitarStopwords(words)
    return words

#Esta version elimina convierte a misnusculas, quita puntuacion, quita stopwords
def preprocessing_text(texto):
    texto.dropna()
    texto.drop_duplicates()
    texto['Textos_espanol'] = texto['Textos_espanol'].apply(contractions.fix)
    texto['words'] = texto['Textos_espanol'].apply(word_tokenize)
    texto['words'] = texto['words'].apply(preProcesamiento)
    texto['words'] = texto['words'].apply(lambda x: ' '.join(map(str, x)))
    return texto['words']

In [313]:
preprocessing_text(data_t)

2651    movilización mujeres manifestaciones callejera...
169     papel combinado comprador proveedor nivel muni...
2993    desempleo particularmente alto jóvenes años im...
1148    además espíritu empresarial vehículo fundament...
2270    aunque sudáfrica oficina condición jurídica so...
                              ...                        
835     compensación enfermedad otorga adultos años mi...
3264    documento cobertura considera fuente secundari...
1653    objetivo análisis identificar puntos común div...
2607    luz aplicación enfoque lente género políticas ...
2732    examinamos resultados ols todas mujeres juntas...
Name: words, Length: 3239, dtype: object

### Clase para la vectorización y transformacion del dataframe

In [315]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

class Transformer_Representacion_Seleccion:
    def __init__(self, count_vectorizer):
        self.count_vectorizer = count_vectorizer
        self.palabras = None
        self.palabras_deseadas = None  # Inicialización de las palabras deseadas

    def fit(self, X, y=None):
        # Ajustar el CountVectorizer y obtener las palabras (características)
        X_transformed = self.count_vectorizer.fit_transform(X)
        self.palabras = self.count_vectorizer.get_feature_names_out()

        # Crear un DataFrame temporal para realizar las selecciones de palabras relevantes
        X_todas_palabras = pd.DataFrame(X_transformed.toarray(), columns=self.palabras)
        
        # Seleccionar las palabras relevantes usando un ciclo tradicional
        self.palabras_deseadas = []
        for nombre in X_todas_palabras.columns:
            # Contar cuántas filas tienen valores distintos de 0 para cada palabra
            dato = (X_todas_palabras[nombre] != 0).sum()
            if dato > 1:
                # Si la palabra aparece en más de una fila, la agregamos a la lista
                self.palabras_deseadas.append(nombre)
        
        return self

    def transform(self, X):
        # Transformar los datos usando CountVectorizer
        X_transformed = self.count_vectorizer.transform(X)
        
        # Crear DataFrame con todas las palabras
        X_todas_palabras = pd.DataFrame(X_transformed.toarray(), columns=self.palabras)
        
        # Seleccionar solo las palabras relevantes
        palabras_a_usar = pd.Index(self.palabras_deseadas).intersection(X_todas_palabras.columns)
        
        # Retornar el DataFrame con las palabras relevantes
        return X_todas_palabras[palabras_a_usar].copy()

### Pipelon ajustado con el procesamiento, transformación y mejores hiperparametros

In [317]:


# ajustamos el pipeline con RandomForestClassifier y los mejores parametros
pipelon = Pipeline([
    ('preprocess', FunctionTransformer(preprocessing_text)),
    ('representacion', Transformer_Representacion_Seleccion(CountVectorizer())),
    ('clf', RandomForestClassifier(max_depth=42, max_features=102, random_state=111)),
])


In [318]:
pipelon.fit(data_t, data_t['sdg'])

Pipeline(steps=[('preprocess',
                 FunctionTransformer(func=<function preprocessing_text at 0x12a9bc220>)),
                ('representacion',
                 <__main__.Transformer_Representacion_Seleccion object at 0x12e9e8500>),
                ('clf',
                 RandomForestClassifier(max_depth=42, max_features=102,
                                        random_state=111))])

### Persistencia del modelo en .joblib

In [320]:
joblib.dump(pipelon, 'pipelon.joblib')

pipelon_cargado = joblib.load('pipelon.joblib')

### Validación con los datos que arroja el modelo de la primera etapa

In [322]:
#validacion
x_test = pd.read_csv("ValidacionX.csv", sep=',')
y_test = pd.read_csv("ValidacionY.csv", sep=',')
y_test = y_test['sdg']

### Metricas

In [324]:
y_pred = pipelon_cargado.predict(data_v)

print("Metricas:\n")
print(classification_report(data_v['sdg'], y_pred))

Metricas:

              precision    recall  f1-score   support

           3       0.99      0.97      0.98       269
           4       0.97      0.98      0.97       266
           5       0.97      0.98      0.97       275

    accuracy                           0.98       810
   macro avg       0.98      0.98      0.98       810
weighted avg       0.98      0.98      0.98       810

